In [ ]:
# can be used to find nearby yelpers and target them with deal and specials, information can be sold to businesses

# win for business, yelp, and yelper

# EDA

In [11]:
import databricks.koalas as ks
import pandas as pd
pd.set_option('display.max_columns', None)
import time
import json
import os



In [2]:
folder = '../data/'

print('here are the df files:\n')
pkl_file_list = []
for file in os.listdir(folder):
    if file.endswith(".pkl"):
        pkl_file_list.append(folder+file)
        print('found ' + file) 

here are the df files:

found tip_df.pkl
found user_df.pkl
found photo_df.pkl
found review_df.pkl
found checkin_df.pkl
found review_sample_df.pkl
found business_df.pkl


In [4]:
business_df = pd.read_pickle('../data/business_df.pkl')
print(business_df.shape)
business_df.head(2)

(192609, 14)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."


In [12]:
treview_df = pd.read_pickle('../data/review_df.pkl')
print(review_df.shape)
review_df.head(2)

(6685900, 9)


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33


In [13]:
user_df = pd.read_pickle('../data/user_df.pkl')
print(user_df.shape)
user_df.head(2)

(1637138, 22)


,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,l6BmjZMeQD3rDxWUbiAiow,Rashmi,95,2013-10-08 23:11:33,84,17,25,"2015,2016,2017","c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g...",5,4.03,2,0,0,0,0,1,1,1,1,2,0
1,4XChL029mKr5hydo79Ljxg,Jenna,33,2013-02-21 22:29:06,48,22,16,,"kEBTgDvFX754S68FllfCaA, aB2DynOxNOJK9st2ZeGTPg...",4,3.63,1,0,0,0,0,0,0,1,1,0,0


In [ ]:
user_df['likes'] = user_df.apply(lambda x: x['useful'] + x['funny']+ x['cool'])
user_df['compliments'] = user_df.apply(lambda x: x['compliment_hot'] + x['compliment_more']+ x['compliment_profile']+ x['compliment_cute']+ x['compliment_list']+ x['compliment_note']+ x['compliment_plain']+ x['compliment_cool']+ x['compliment_funny']+ x['compliment_writer']+ x['compliment_photos'])
user_df['elite_years'] = user_df.apply(lambda x: len(x['elite']))

In [ ]:
user_df.drop['useful', 'funny', 'cool',  ]

In [7]:
checkin_df = pd.read_pickle('../data/checkin_df.pkl')
print(checkin_df.shape)
checkin_df.head(2)

(161950, 2)


,business_id,date
0,--1UhMGODdWsrMastO9DZw,"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016..."
1,--6MefnULPED_I942VcFNA,"2011-06-04 18:22:23, 2011-07-23 23:51:33, 2012..."


In [8]:
tip_df = pd.read_pickle('../data/tip_df.pkl')
print(tip_df.shape)
tip_df.head(2)

(1223094, 5)


,user_id,business_id,text,date,compliment_count
0,UPw5DWs_b-e2JRBS-t37Ag,VaKXUpmWTTWDKbpJ3aQdMw,"Great for watching games, ufc, and whatever el...",2014-03-27 03:51:24,0
1,Ocha4kZBHb4JK0lOWvE0sg,OPiPeoJiv92rENwbq76orA,Happy Hour 2-4 daily with 1/2 price drinks and...,2013-05-25 06:00:56,0


In [7]:
user_df[user_df['user_id'] == 'g57_SLLN3jASzZcafLMdtw']

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos


In [5]:
os.stat('../data/review_df.pkl')

os.stat_result(st_mode=33188, st_ino=12893316484, st_dev=16777220, st_nlink=1, st_uid=501, st_gid=20, st_size=5148054604, st_atime=1563206913, st_mtime=1562786435, st_ctime=1562863771)

In [6]:
os.path.getsize('../data/review_df.pkl')

5148054604

In [10]:
pd.set_option('display.max_columns', None)
df = review_df.merge(business_df, left_on='business_id', right_on='business_id').merge(user_df, left_on='user_id', right_on='user_id')

df.head()

,review_id,user_id,business_id,stars_x,useful_x,funny_x,cool_x,text,date,name_x,address,city,state,postal_code,latitude,longitude,stars_y,review_count_x,is_open,attributes,categories,hours,name_y,review_count_y,yelping_since,useful_y,funny_y,cool_y,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,MountainView Hospital,3100 N Tenaya Way,Las Vegas,NV,89128,36.215546,-115.248798,2.5,238,1,"{'ByAppointmentOnly': 'False', 'GoodForKids': ...","Fitness & Instruction, Doctors, Health & Medic...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",Stefan,10,2008-07-10 17:24:47,18,4,0,,frQs7y5qa-X1pvAM0sJe1w,0,2.0,0,0,0,0,0,0,0,0,0,0,0
1,oy8f3bxyl7zZJFDQ5edtIA,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,3,0,0,6 plus hours in the ER on a Tuesday Morning gr...,2013-03-27 14:17:13,MountainView Hospital,3100 N Tenaya Way,Las Vegas,NV,89128,36.215546,-115.248798,2.5,238,1,"{'ByAppointmentOnly': 'False', 'GoodForKids': ...","Fitness & Instruction, Doctors, Health & Medic...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",Stefan,10,2008-07-10 17:24:47,18,4,0,,frQs7y5qa-X1pvAM0sJe1w,0,2.0,0,0,0,0,0,0,0,0,0,0,0
2,nsr-0uYDWv22qFdARRa2CQ,hG7b0MtEbXx5QzbzE6C_VA,lsAe6PMRTVGtaBqw3zKlMQ,2.0,0,0,0,I'll give it a 2 because it is still a WM with...,2013-04-24 09:04:10,Walmart Supercenter,3950 W Lake Mead Blvd,North Las Vegas,NV,89032,36.201056,-115.191736,2.0,56,1,"{'RestaurantsTakeOut': 'True', 'Caters': 'True...","Discount Store, Food, Drugstores, Electronics,...","{'Monday': '6:0-0:0', 'Tuesday': '6:0-0:0', 'W...",Stefan,10,2008-07-10 17:24:47,18,4,0,,frQs7y5qa-X1pvAM0sJe1w,0,2.0,0,0,0,0,0,0,0,0,0,0,0
3,D4_FpPSPRNtAj1964vXFPQ,hG7b0MtEbXx5QzbzE6C_VA,xpEq8x5caXFm2lmjQ51rVQ,2.0,2,0,0,Unable to get their over the air signal. So wh...,2014-02-27 04:20:15,KLAS-TV,3228 Channel 8 Dr,Las Vegas,NV,89109,36.130689,-115.161787,3.0,10,1,None,"Television Stations, Mass Media","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",Stefan,10,2008-07-10 17:24:47,18,4,0,,frQs7y5qa-X1pvAM0sJe1w,0,2.0,0,0,0,0,0,0,0,0,0,0,0
4,HeTRGukwBk1TPoX0dr4xZA,hG7b0MtEbXx5QzbzE6C_VA,Y07qIurPKiUJDnTZWxZN2A,5.0,0,0,0,Not a huge selection but what they do have is ...,2012-08-28 23:34:11,Bistro Buffet,4321 W Flamingo Rd,Las Vegas,NV,89103,36.115137,-115.195433,3.0,433,0,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Hotels & Travel, Hotels, Restaurants, Italian,...","{'Monday': '7:0-22:0', 'Tuesday': '7:0-22:0', ...",Stefan,10,2008-07-10 17:24:47,18,4,0,,frQs7y5qa-X1pvAM0sJe1w,0,2.0,0,0,0,0,0,0,0,0,0,0,0


## save to pickle

In [13]:
df.to_pickle('../data/full_df.pkl')

KeyboardInterrupt: 

In [9]:
business_df.city.value_counts()

Las Vegas                     29370
Toronto                       18906
Phoenix                       18766
Charlotte                      9509
Scottsdale                     8837
Calgary                        7736
Pittsburgh                     7017
Montréal                       6449
Mesa                           6080
Henderson                      4892
Tempe                          4550
Chandler                       4309
Cleveland                      3605
Glendale                       3543
Madison                        3494
Gilbert                        3462
Mississauga                    3112
Peoria                         1919
Markham                        1766
North Las Vegas                1548
Champaign                      1262
North York                     1205
Surprise                       1148
Scarborough                    1103
Richmond Hill                  1026
Brampton                       1002
Concord                         990
Vaughan                     

In [21]:
df.groupby(['state', 'city']).count().sort_values(by=['business_id'], ascending=False)

review_id  user_id  business_id  stars_x  \
state city                                                                  
NV    Las Vegas                    2030062  2030062      2030062  2030062   
AZ    Phoenix                       734105   734105       734105   734105   
ON    Toronto                       525422   525422       525422   525422   
AZ    Scottsdale                    387550   387550       387550   387550   
NC    Charlotte                     309406   309406       309406   309406   
PA    Pittsburgh                    225489   225489       225489   225489   
NV    Henderson                     219880   219880       219880   219880   
AZ    Tempe                         202074   202074       202074   202074   
      Mesa                          168311   168311       168311   168311   
      Chandler                      157569   157569       157569   157569   
QC    Montréal                      151238   151238       151238   151238   
AZ    Gilbert                       129028   129028       129028   129028   
OH    Cleveland                     114389   114389       114389   114389   
WI    Madison                       104634   104634       104634   104634   
AZ    Glendale                      100299   100299       100299   100299   
AB    Calgary                        97647    97647        97647    97647   
AZ    Peoria                         57857    57857        57857    57857   
ON    Mississauga                    55585    55585        55585    55585   
      Markham                        51296    51296        51296    51296   
NV    North Las Vegas                50937    50937        50937    50937   
AZ    Surprise                       34877    34877        34877    34877   
IL    Champaign                      30904    30904        30904    30904   
AZ    Goodyear                       28596    28596        28596    28596   
ON    Richmond Hill                  24242    24242        24242    24242   
AZ    Avondale                       22133    22133        22133    22133   
ON    North York                     20353    20353        20353    20353   
      Scarborough                    19115    19115        19115    19115   
OH    Lakewood                       18212    18212        18212    18212   
NC    Concord                        16672    16672        16672    16672   
ON    Vaughan                        16126    16126        16126    16126   
...                                    ...      ...          ...      ...   
QC    Ste-Therese-de-Blainville          3        3            3        3   
      St-Jean-sur-Richelieu              3        3            3        3   
NV    Los Vegas                          3        3            3        3   
      Nellis                             3        3            3        3   
      las Vegas                          3        3            3        3   
QC    Vieux-Montréal                     3        3            3        3   
PA    Cranberry Township                 3        3            3        3   
      Chateau                            3        3            3        3   
      CARSON                             3        3            3        3   
NV    Hnederson                          3        3            3        3   
QC    St-Clet                            3        3            3        3   
AZ    Maricopa                           3        3            3        3   
NC    Catawba Springs                    3        3            3        3   
IL    Ivesdale                           3        3            3        3   
      Mansfield                          3        3            3        3   
      St. Joseph                         3        3            3        3   
QC    Saint-Roch-de-l'Achigan            3        3            3        3   
IL    Thomasboro                         3        3            3        3   
QC    Saint-laurent                      3        3            3        3   
PA    Port Vue               

In [14]:
df.groupby('city').count().sort_values(by=['business_id'], ascending=False)

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,address,state,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours
city,,,,,,,,,,,,,,,,,,,,,
Las Vegas,2030798,2030798,2030798,2030798,2030798,2030798,2030798,2030798,2030798,2030798,2030798,2030798,2030798,2030798,2030798,2030798,2030798,2030798,1963851,2030467,1911201
Phoenix,734136,734136,734136,734136,734136,734136,734136,734136,734136,734136,734136,734136,734136,734136,734136,734136,734136,734136,698854,733842,673848
Toronto,525437,525437,525437,525437,525437,525437,525437,525437,525437,525437,525437,525437,525437,525437,525437,525437,525437,525437,505736,525320,467043
Scottsdale,387550,387550,387550,387550,387550,387550,387550,387550,387550,387550,387550,387550,387550,387550,387550,387550,387550,387550,375421,387499,363583
Charlotte,309425,309425,309425,309425,309425,309425,309425,309425,309425,309425,309425,309425,309425,309425,309425,309425,309425,309425,298613,309351,285910
Pittsburgh,225496,225496,225496,225496,225496,225496,225496,225496,225496,225496,225496,225496,225496,225496,225496,225496,225496,225496,218200,225457,206877
Henderson,219880,219880,219880,219880,219880,219880,219880,219880,219880,219880,219880,219880,219880,219880,219880,219880,219880,219880,210534,219817,205225
Tempe,202074,202074,202074,202074,202074,202074,202074,202074,202074,202074,202074,202074,202074,202074,202074,202074,202074,202074,195416,202001,190231
Mesa,168311,168311,168311,168311,168311,168311,168311,168311,168311,168311,168311,168311,168311,168311,168311,168311,168311,168311,160198,168217,154911
